<a href="https://colab.research.google.com/github/kbw612/Fantasy/blob/main/colab/Contest_Player_Stacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Default title text
import pandas as pd

name_column_name = 'Name'
position_column_name = 'Position'
salary_column_name = 'Salary'
id_column_name = 'ID'

class PlayerStack:
    def __init__(self, total_salary, players):
        self.total_salary = total_salary
        self.players = players

class Player:
    def __init__(self, name, position, salary):
        self.position = position
        self.name = name
        self.salary = salary

def get_contest_players(week):
  df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/DKSalaries-week{week}.csv")

  return df

def get_receivers(week):
  df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/DK-receivers-main-slate-week{week}.csv")

  return df

def get_running_backs(week):
  df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/DK-RBs-main-slate-week{week}.csv")

  return df

def get_team_players(players_df, team_abbrev):
  wr_position_condition =  players_df["Position"].isin(['WR'])
  rb_position_condition = players_df["Position"].isin(['RB'])
  te_position_condition = players_df["Position"].isin(['TE'])
  team_condition = players_df["TeamAbbrev"].isin([team_abbrev])

  wrs_df = players_df[team_condition & wr_position_condition].head(4)
  rbs_df = players_df[team_condition & rb_position_condition].head(2)
  tes_df = players_df[team_condition & te_position_condition].head(1)

  team_players_df = pd.concat([wrs_df, rbs_df, tes_df], ignore_index=True)

  return team_players_df

def get_player(players_df, player_name):
  filtered_df = players_df[players_df[name_column_name] == player_name]

  if len(filtered_df) > 0:
    return Player(filtered_df[name_column_name].iloc[0], filtered_df[position_column_name].iloc[0], filtered_df[salary_column_name].iloc[0])

  return Player('', '', 0)

def get_player_stacks(players_df):
  player_stacks = []
  for rowOuter in players_df.index:
    for rowInner in players_df.index:
      if players_df.at[rowOuter, name_column_name] != players_df.at[rowInner, name_column_name]:
        player_stack = []
        #salaryOuter = players_df.at[rowOuter, salary_column_name]

        player_stack.append(players_df.at[rowOuter, name_column_name])
        player_stack.append(players_df.at[rowInner, name_column_name])

        sorted_players = player_stack.sort()

        #print(player_stack)

        if player_stack not in player_stacks:
          player_stacks.append(player_stack)

  return player_stacks

def build_player_stacks(all_players_df, player_stacks):
  player_stacks2 = []
  for player_stack in player_stacks:
    total_salary = 0
    players = []

    for player_name in player_stack:
      player = get_player(all_players_df, player_name)
      total_salary += player.salary
      players.append(player)

    players = sorted(players, key=lambda x: x.salary, reverse=True)

    player_stack_obj = PlayerStack(total_salary, players)
    player_stacks2.append(player_stack_obj)

  return sorted(player_stacks2, key=lambda x: x.total_salary, reverse=True)

def build_stacks_df(player_stacks):
  stacks_df = pd.DataFrame(columns=[name_column_name, position_column_name, salary_column_name])
  for player_stack in player_stacks:
    for player in player_stack.players:
      row = {name_column_name: [player.name], position_column_name: [player.position], salary_column_name: [player.salary]}
      stacks_df = pd.concat([stacks_df, pd.DataFrame(row)], axis=0, ignore_index=True)

    row = {name_column_name: [''], position_column_name: [''], salary_column_name: [player_stack.total_salary]}
    stacks_df = pd.concat([stacks_df, pd.DataFrame(row)], axis=0, ignore_index=True)

    row = {name_column_name: [''], position_column_name: [''], salary_column_name: ['']}
    stacks_df = pd.concat([stacks_df, pd.DataFrame(row)], axis=0, ignore_index=True)

  return stacks_df

#positions = ["RB", "WR", "TE"]
#teams = ['IND', 'TB']
#team1 = ['IND']
#team2 = ['TB']

#print(get_team_players(players_df, 'IND'))
#print(get_team_players(players_df, 'TB'))

#player1,player2,player3

week = 15
all_players_df = get_contest_players(week)
receivers_df = get_receivers(week)
running_backs_df = get_running_backs(week)

player_stacks = get_player_stacks(receivers_df)
player_stacks2 = build_player_stacks(all_players_df, player_stacks)
stacks_df = build_stacks_df(player_stacks2)

with pd.option_context("display.max_rows", 100000):
  print(stacks_df.to_string(index=False))
  print("")

               Name Position Salary
   Justin Jefferson       WR   8100
  Amon-Ra St. Brown       WR   7900
                              16000
                                   
   Justin Jefferson       WR   8100
      Ja'Marr Chase       WR   7600
                              15700
                                   
  Amon-Ra St. Brown       WR   7900
      Ja'Marr Chase       WR   7600
                              15500
                                   
   Justin Jefferson       WR   8100
Michael Pittman Jr.       WR   7300
                              15400
                                   
  Amon-Ra St. Brown       WR   7900
Michael Pittman Jr.       WR   7300
                              15200
                                   
      Ja'Marr Chase       WR   7600
Michael Pittman Jr.       WR   7300
                              14900
                                   
   Justin Jefferson       WR   8100
   Courtland Sutton       WR   6400
                            